# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Install requirements
Let's start by installing and importing library dependencies. We'll use requests to interact with the BioNeMo service, BioPython to parse FASTA sequences into SeqRecord objects, scikit-learn for classification tasks, and matplotlib for visualization.

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [2]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [3]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`.

In [4]:
n_clients = 3
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0
out_dir = "/tmp/fasta/mixed_soft"

import os
import re
import numpy as np
import pandas as pd
import uuid
import random
random.seed(seed)
        
# extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequence"] = str(x.seq)
        #entry["id"] = '6' + str(uuid.uuid4())[:5]
        entry["id"] = str(i)
    
        data.append(entry)
        
print(f"Read {len(data)} valid sequences.")
        
# shuffle & split training data
random.shuffle(data)
data_splits = np.array_split(data, n_clients)

# save split data
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# save split training data
for idx, split in enumerate(data_splits):
    client_name = f"site-{idx+1}"
    
    split_dict = {}
    for entry in split:
        for k, v in entry.items():
            if k not in split_dict:
                split_dict[k] = []
            else:
                split_dict[k].append(v)
    df = pd.DataFrame(split_dict)
    
    #df.to_csv(os.path.join(out_dir, f"data_{client_name}.csv"), index=False, lineterminator='\n', columns=["id", "sequence"])#, "TARGET", "SET"])#, )
    df.to_csv(os.path.join(out_dir, f"data_{client_name}.csv"), index=False, columns=["id", "sequence", "TARGET", "SET"])
    #with open(os.path.join(out_dir, f"data_{client_name}.csv"), "w") as f:
    #    f.write("id,sequence\r\n")
    #    for entry in split:
    #        f.write(f"{entry['id']},{entry['sequence']}\r\n")
        
    # DEBUG
    #df = pd.read_csv("/tmp/data/FLIP/secondary_structure/test/x000.csv")
    #df.to_csv(os.path.join(out_dir, f"data_{client_name}.csv"), index=False)
        
    print(f"Saving {len(split)} training sequences for client {client_name}.")


Read 8619 valid sequences.
Saving 2873 training sequences for client site-1.
Saving 2873 training sequences for client site-2.
Saving 2873 training sequences for client site-3.


### Federated embedding extraction
Running inference of the ESM-1nv model to extract embeddings requires a GPU with at least 12 GB memory. Here we run inference on each client sequentially using one thread to preserve GPU memory.

In [5]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=n_clients,
    threads=1  # due to memory constraints, we run the client execution sequentially in one thread
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-07-28 20:58:48,642 - SimulatorRunner - INFO - Create the Simulator Server.
2023-07-28 20:58:48,650 - Cell - INFO - server: creating listener on tcp://0:50995
2023-07-28 20:58:48,681 - Cell - INFO - server: created backbone external listener for tcp://0:50995
2023-07-28 20:58:48,683 - ConnectorManager - INFO - 40643: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-07-28 20:58:48,687 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:25196] is starting
2023-07-28 20:58:49,191 - Cell - INFO - server: created backbone internal listener for tcp://localhost:25196
2023-07-28 20:58:49,194 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:50995] is starting
2023-07-28 20:58:49,287 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 38489
2023-07-28 20:58:49,288 - SimulatorRunner - INFO - Deploy the Apps.
2023-07-28 20:58:50,067 - SimulatorRunner - INFO - Create the simulat

[NeMo W 2023-07-28 20:59:06 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-07-28 20:59:07,069 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=a321ed40-0cc8-492e-a327-1f0963a5e7fa]: assigned task to client site-1: name=bionemo_inference, id=a321ed40-0cc8-492e-a327-1f0963a5e7fa
2023-07-28 20:59:07,071 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=a321ed40-0cc8-492e-a327-1f0963a5e7fa]: sent task assignment to client. client_name:site-1 task_id:a321ed40-0cc8-492e-a327-1f0963a5e7fa
2023-07-28 20:59:07,073 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: bionemo_inference   task_id: a321ed40-0cc8-492e-a327-1f0963a5e7fa  sharable_header_task_id: a321ed40-0cc8-492e-a327-1f0963a5e7fa


[NeMo W 2023-07-28 20:59:06 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2023-07-28 20:59:07 utils:250] Restoring model from /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/models/esm1nv.nemo
[NeMo I 2023-07-28 20:59:07 utils:254] Loading model class: bionemo.model.protein.esm1nv.esm1nv_model.ESM1nvModel
2023-07-28 20:59:07,163 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-07-28 20:59:07,163 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2023-07-28 20:59:07,163 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2023-07-28 20:59:07,163 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
[NeMo I 2023-07-28 20:59:07 exp_manager:374] Experiments will be logged at /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/nemo_experiments/ESM1nv_Inference/2023-07-28_20-59-07
[NeMo I 2023-07-28 20:59:07 exp_manager:797] TensorboardLogger has been set up
[NeMo I 2023-07-28 20:59:07 utils:234] 
   

...

Predicting DataLoader 0: 100%|██████████| 23/23 [00:28<00:00,  1.24s/it]2023-07-28 21:01:37,555 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job]: got result from client site-3 for task: name=bionemo_inference, id=acc03bd1-7ebe-47be-b434-0aa44e35afee
2023-07-28 21:01:37,558 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job, peer_rc=OK, task_name=bionemo_inference, task_id=acc03bd1-7ebe-47be-b434-0aa44e35afee]: finished processing client result by bionemo_inference
2023-07-28 21:01:37,560 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-3   task_id:acc03bd1-7ebe-47be-b434-0aa44e35afee
2023-07-28 21:01:37,565 - SimulatorClientRunner - INFO - Simulate Run client: site-1 on GPU group: None
2023-07-28 21:01:37,732 - BioNeMoInference - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: task bi

[NeMo W 2023-07-28 21:01:49 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-07-28 21:01:50,431 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job]: server runner is finalizing - asked client to end the run
2023-07-28 21:01:50,433 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: __end_run__   task_id:   sharable_header_task_id: 
2023-07-28 21:01:50,442 - SimulatorClientRunner - INFO - Simulate Run client: site-2 on GPU group: None
2023-07-28 21:01:50,445 - FederatedClient - INFO - Shutting down client run: site-1
2023-07-28 21:01:50,447 - FederatedClient - INFO - Shutting down client run: site-2
2023-07-28 21:01:50,448 - FederatedClient - INFO - Shutting down client run: site-3
2023-07-28 21:01:50,449 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: asked to abort - triggered abort_signal to stop the RUN


[NeMo W 2023-07-28 21:01:50 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-07-28 21:01:53,983 - MPM - INFO - MPM: Good Bye!
Simulator finished with run_status 0


### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo server are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM-1nv, `D=768`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [6]:
# load embeddings from site-1
import pickle
protein_embeddings = pickle.load(open(os.path.join(out_dir, "data_site-1.pkl"), "rb"))
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    protein_embedding = protein_embeddings[i]
    print(f"Inference result contains {list(protein_embedding.keys())}")
    x = protein_embedding["embeddings"]
    print(f"{protein_embedding['id']}: range {np.min(x)}-{np.max(x)}, mean={np.mean(x)}, shape={x.shape}")

Loaded 2862 embeddings from site-1.
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
10649: range -0.90283203125-1.7978515625, mean=-0.0032165292650461197, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
7598: range -0.8017578125-1.2138671875, mean=-0.0010715207317844033, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
11506: range -0.86865234375-1.708984375, mean=-0.0034913886338472366, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
7116: range -0.7880859375-1.5556640625, mean=-0.0015334287891164422, shape=(768,)


Let's enumerate the labels corresponding to potential subcellular locations.

In [7]:
# Let's also print all the labels

labels = set([entry['TARGET'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

1. Lysosome
2. Extracellular
3. Plastid
4. Cytoplasm
5. Peroxisome
6. Endoplasmic reticulum
7. Golgi apparatus
8. Nucleus
9. Cell membrane
10. Mitochondrion


### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier. The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the call below, we define the MLP to use the Adam optimizer with a network of 32 hidden layers, defining a random state (or seed) for reproducibility, and trained for a maximum of 500 iterations.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace="/tmp/nvflare/bionemo/fedavg",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-07-28 23:06:38,856 - SimulatorRunner - INFO - Create the Simulator Server.
2023-07-28 23:06:38,861 - Cell - INFO - server: creating listener on tcp://0:35581
2023-07-28 23:06:38,886 - Cell - INFO - server: created backbone external listener for tcp://0:35581
2023-07-28 23:06:38,887 - ConnectorManager - INFO - 106135: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-07-28 23:06:38,890 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:34259] is starting
2023-07-28 23:06:39,393 - Cell - INFO - server: created backbone internal listener for tcp://localhost:34259
2023-07-28 23:06:39,396 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:35581] is starting
2023-07-28 23:06:39,487 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 33631
2023-07-28 23:06:39,488 - SimulatorRunner - INFO - Deploy the Apps.
2023-07-28 23:06:39,496 - SimulatorRunner - INFO - Create the simula

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


2023-07-28 23:06:43,064 - BioNeMoMLPModelPersistor - INFO - [identity=simulator_server, run=simulate_job]: MLPClassifier coefficients
2023-07-28 23:06:43,072 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2023-07-28 23:06:43,075 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job]: Initializing ScatterAndGather workflow.
2023-07-28 23:06:43,082 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2023-07-28 23:06:43,093 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Beginning ScatterAndGather training phase.
2023-07-28 23:06:43,100 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Round 0 started.
20